<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a5141327cd50682c66d76d6a9cccbc9a2bc014b8e070ce4907dacf709020307c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 16:19:08
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 64.24 K (0.46%)
Current PnL: -19.76 L (-13.16%)
CY Booked + Current PnL: -8.32 L (-5.54%)
-------------------
Total profit:  1.84 L
Total loss:  -21.60 L
-------------------
Total Booked + Current PnL: 18.66 L (15.15%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.27 L (61.53%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118879.0,18375.0,10735.0,-2.29,18.28,9.03,28.96,79.0,1.71,0.84,58.46,MH,ATH,METALS
14,BLUESTARCO,2080.00,12.85,59.0,X-MC,3.22,194910.0,30240.0,13098.0,3.00,18.36,6.72,26.31,89.0,2.31,1.38,27.81,X40N,NTT,AC
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.68,213967.0,9467.0,13822.0,3.22,4.63,6.46,11.39,10.0,0.68,1.51,14.00,X40N,ATH,PHARMA
84,VOLTAS,1530.00,1.30,61.0,X-MC,1.09,212595.0,20853.0,16901.0,2.55,10.88,7.95,19.69,99.0,1.23,1.51,18.41,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.01,83218.0,-17559.0,17651.0,-0.28,-17.42,21.21,0.09,245.0,-0.99,0.59,9.33,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-6.33,41.0,H-SC,6.22,111535.0,-13762.0,72687.0,4.93,-10.98,65.17,47.03,141.0,-0.19,0.79,39.65,MH,ATH,POWER
10,BANDHANBNK,400.00,-9.22,59.0,H-SC,2.93,226889.0,-51670.0,309114.0,3.69,-18.55,136.24,92.42,151.0,-0.17,1.61,31.12,XY24,NTT,BANKS
63,SFL,1287.00,33.01,48.0,M-SC,7.19,175724.0,-86494.0,156324.0,3.46,-32.99,88.96,26.63,229.0,-0.55,1.24,9.26,XY24,NTT,MISC
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.32,84691.0,-28858.0,68905.0,3.30,-25.41,81.36,35.27,268.0,-0.42,0.60,107.28,XR,NTT,HOTELS
19,CIPLA,1662.74,-19.78,58.0,X-LC,3.68,213967.0,9467.0,13822.0,3.22,4.63,6.46,11.39,10.0,0.68,1.51,14.00,X40N,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,101.92,26.0,H-SC,7.94,136739.0,-81757.0,109952.0,-4.79,-37.42,80.41,12.90,153.0,-0.74,0.97,4.08,OX40N,NTT,FINANCE
60,SAIL,228.00,44.35,50.0,M-MC,12.66,227597.0,2635.0,165031.0,-3.14,1.17,72.51,74.53,192.0,0.02,1.61,33.80,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-24.05,45.0,H-MC,6.65,102742.0,-29093.0,71313.0,-2.97,-22.07,69.41,32.03,98.0,-0.41,0.73,15.08,OX40N,NTT,IT
32,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.53,31.17,X5K,ATH,METALS
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118879.0,18375.0,10735.0,-2.29,18.28,9.03,28.96,79.0,1.71,0.84,58.46,MH,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,64.0,H-SC,13.79,131706.0,-10773.0,30292.0,1.18,-7.56,23.00,13.70,163.0,-0.36,0.93,55.45,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,52.0,H-SC,1.95,222750.0,-46917.0,83242.0,0.88,-17.40,37.37,13.47,138.0,-0.56,1.58,13.26,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.01,83218.0,-17559.0,17651.0,-0.28,-17.42,21.21,0.09,245.0,-0.99,0.59,9.33,OX40N,NTT,DURABLES
18,CERA,9475.0,-24.82,33.0,H-SC,1.82,137241.0,-38662.0,80684.0,-0.93,-21.98,58.79,23.89,149.0,-0.48,0.97,18.53,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.05,45.0,H-MC,6.65,102742.0,-29093.0,71313.0,-2.97,-22.07,69.41,32.03,98.0,-0.41,0.73,15.08,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,50.0,M-MC,12.66,227597.0,2635.0,165031.0,-3.14,1.17,72.51,74.53,192.0,0.02,1.61,33.80,XY24,BTT,STEEL
32,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.53,31.17,X5K,ATH,METALS
86,WIPRO,420.00,-12.43,55.0,M-LC,5.71,154194.0,3249.0,106209.0,0.93,2.15,68.88,72.51,53.0,0.03,1.09,8.30,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,57.0,M-LC,9.18,158350.0,17510.0,26983.0,1.04,12.43,17.04,31.59,66.0,0.65,1.12,42.21,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.94,194972.0,15164.0,97252.0,0.78,8.43,49.88,62.52,88.0,0.16,1.38,45.40,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118879.0,18375.0,10735.0,-2.29,18.28,9.03,28.96,79.0,1.71,0.84,58.46,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.94,194972.0,15164.0,97252.0,0.78,8.43,49.88,62.52,88.0,0.16,1.38,45.40,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.53,31.17,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,42.0,H-SC,8.73,126744.0,3408.0,128214.0,-0.86,2.76,101.16,106.72,119.0,0.03,0.90,26.79,AR,ATH,MISC
86,WIPRO,420.00,-12.43,55.0,M-LC,5.71,154194.0,3249.0,106209.0,0.93,2.15,68.88,72.51,53.0,0.03,1.09,8.30,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.42,33.0,X-LC,5.54,205938.0,-45830.0,99036.0,0.16,-18.20,48.09,21.13,27.0,-0.46,1.46,10.77,X40,ATH,PAINTS
23,DABUR,735.00,-12.30,36.0,X-MC,3.75,237504.0,-10564.0,118966.0,0.89,-4.26,50.09,43.70,102.0,-0.09,1.68,9.57,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53710.0,-11296.0,162226.0,3.05,-17.38,302.04,232.18,198.0,-0.07,0.38,3.05,XY24,NTT,MISC
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.55,81705.0,-63455.0,139381.0,-0.75,-43.71,170.59,52.31,136.0,-0.46,0.58,9.37,X40N,NTT,FOOTWEAR
29,HAVELLS,2069.16,-5.90,39.0,X-MC,0.57,233360.0,-14371.0,89424.0,0.68,-5.80,38.32,30.30,92.0,-0.16,1.65,6.72,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-5.90,39.0,X-MC,0.57,233360.0,-14371.0,89424.0,0.68,-5.80,38.32,30.30,92.0,-0.16,1.65,6.72,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,1.30,61.0,X-MC,1.09,212595.0,20853.0,16901.0,2.55,10.88,7.95,19.69,99.0,1.23,1.51,18.41,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,43.0,X-LC,1.59,246459.0,-30064.0,140137.0,-0.32,-10.87,56.86,39.81,54.0,-0.21,1.74,18.12,XY24,NTT,AUTO
65,SIEMENS,4671.50,-2.18,46.0,H-LC,1.66,157370.0,-28725.0,76199.0,-0.98,-15.44,48.42,25.51,15.0,-0.38,1.11,15.97,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-19.88,44.0,X-MC,1.80,137847.0,1714.0,29417.0,-1.06,1.26,21.34,22.86,91.0,0.06,0.98,14.03,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.40,39.0,X-LC,10.36,283049.0,-32793.0,145459.0,-0.50,-10.38,51.39,35.67,5.0,-0.23,2.00,2.03,X40N,ATH,FMCG
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53710.0,-11296.0,162226.0,3.05,-17.38,302.04,232.18,198.0,-0.07,0.38,3.05,XY24,NTT,MISC
22,COLPAL,3767.14,-4.54,40.0,X-MC,7.26,222780.0,-40585.0,153941.0,0.87,-15.41,69.10,43.04,84.0,-0.26,1.58,3.43,XY25,ATH,FMCG
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196969.0,-3169.0,24050.0,0.73,-1.58,12.21,10.44,4.0,-0.13,1.39,3.89,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.48,39.0,X-MC,4.96,180400.0,-20106.0,117260.0,-0.08,-10.03,65.00,48.45,90.0,-0.17,1.28,4.03,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,37.0,X-SC,37.75,53710.0,-11296.0,162226.0,3.05,-17.38,302.04,232.18,198.0,-0.07,0.38,3.05,XY24,NTT,MISC
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.55,81705.0,-63455.0,139381.0,-0.75,-43.71,170.59,52.31,136.0,-0.46,0.58,9.37,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.15,42.0,X-SC,4.43,97027.0,-31643.0,76942.0,-0.28,-24.59,79.30,35.20,219.0,-0.41,0.69,11.37,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.94,51.0,X-SC,1.45,108102.0,6098.0,37825.0,0.77,5.98,34.99,43.06,122.0,0.16,0.77,24.12,X40N,ATH,MISC
33,HONAUT,58357.33,-22.39,49.0,X-SC,2.12,109425.0,-18513.0,65644.0,0.65,-14.47,59.99,36.84,143.0,-0.28,0.77,12.02,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,51.0,X-LC,12.16,284660.0,-61296.0,129549.0,-1.09,-17.72,45.51,19.73,1.0,-0.47,2.02,4.84,X40,ATH,IT
40,INFY,2275.00,-17.09,58.0,X-LC,7.78,322674.0,9786.0,161886.0,0.37,3.13,50.17,54.87,3.0,0.06,2.28,9.98,X40,BTT,IT
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196969.0,-3169.0,24050.0,0.73,-1.58,12.21,10.44,4.0,-0.13,1.39,3.89,X40,NTT,FMCG
83,VBL,671.64,-21.40,39.0,X-LC,10.36,283049.0,-32793.0,145459.0,-0.50,-10.38,51.39,35.67,5.0,-0.23,2.00,2.03,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,47.0,H-LC,7.50,248928.0,-12691.0,131907.0,0.77,-4.85,52.99,45.57,7.0,-0.10,1.76,5.92,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.32,84691.0,-28858.0,68905.0,3.30,-25.41,81.36,35.27,268.0,-0.42,0.60,107.28,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,45.0,H-SC,10.41,91440.0,-6540.0,28145.0,1.20,-6.67,30.78,22.05,152.0,-0.23,0.65,32.91,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118879.0,18375.0,10735.0,-2.29,18.28,9.03,28.96,79.0,1.71,0.84,58.46,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,64.0,H-SC,13.79,131706.0,-10773.0,30292.0,1.18,-7.56,23.00,13.70,163.0,-0.36,0.93,55.45,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,15.38,145998.0,-3222.0,147400.0,0.72,-2.16,100.96,96.62,208.0,-0.02,1.03,58.00,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,67.0,H-SC,9.06,154030.0,-25041.0,90770.0,1.62,-13.98,58.93,36.71,135.0,-0.28,1.09,52.83,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,15.38,145998.0,-3222.0,147400.0,0.72,-2.16,100.96,96.62,208.0,-0.02,1.03,58.00,XY25,NTT,FINANCE
32,HINDZINC,730.22,32.99,64.0,M-LC,7.80,216760.0,11684.0,100880.0,-2.72,5.70,46.54,54.89,52.0,0.12,1.53,31.17,X5K,ATH,METALS
26,GREENPANEL,537.00,238.50,62.0,M-SC,4.73,151182.0,-31896.0,110877.0,1.56,-17.42,73.34,43.14,230.0,-0.29,1.07,42.21,XY24,NTT,MISC
68,STARHEALTH,761.00,16.85,68.0,H-SC,7.42,260440.0,-20683.0,153555.0,0.03,-7.36,58.96,47.26,144.0,-0.13,1.84,39.54,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.43
2,50,75.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.02
LC    32.24
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.99
X40      14.61
X40N     11.26
XR       11.00
XY25     10.44
AR        8.14
OX40N     7.56
X200      1.74
MH        1.63
X5K       1.53
SR        1.10
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.48
X-LC    22.88
X-MC    17.22
M-SC    12.56
H-LC     4.61
H-MC     4.60
X-SC     4.53
M-LC     3.74
M-MC     1.61
L-SC     1.45
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-5.52,40.30
IT,12.82,-16.05,78.91
FINANCE,10.47,-14.23,69.39
MISC,6.95,-17.16,82.07
BANKS,6.44,-9.73,67.46
PAINTS,5.49,-19.02,36.76
ELECTRICAL,5.29,-9.92,48.48
INSURANCE,3.78,-3.69,42.45
AC,3.58,3.98,12.10


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3194827.0,22
XR,1356138.0,14
AR,1180528.0,9
X40,777379.0,10
X40N,645123.0,9
OX40N,579397.0,10
XY25,464666.0,7
SR,265348.0,2
X5K,100880.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3401941.0,25
M-SC,1268240.0,15
X-MC,1103737.0,12
X-LC,1008379.0,13
X-SC,541852.0,6
H-MC,359678.0,4
H-LC,287240.0,3
L-SC,255602.0,3
M-LC,234072.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188203.0      6
           AR         807537.0      5
           XR         795534.0      7
M-SC       XY24       678050.0      6
X-MC       XY24       586003.0      4
X-LC       X40        483903.0      5
X-SC       X40N       302684.0      4
H-SC       OX40N      272632.0      4
           SR         265348.0      2
X-MC       X40        216534.0      4
X-LC       X40N       212081.0      3
H-LC       AR         208106.0      2
X-LC       XY24       207953.0      2
H-MC       XY24       180378.0      1
X-MC       XY25       170842.0      2
L-SC       XR         166984.0      2
M-MC       XY24       165031.0      1
M-SC       AR         164885.0      2
X-SC       XY24       162226.0      1
M-SC       XY25       147400.0      1
           OX40N      146834.0      4
           XR         131071.0      2
X-MC       X40N       130358.0      2
M-LC       XR         106209.0      1
X-LC       XY25       104442.0      3
M-LC       X5K        100880.0      1
H-MC       XR          97252.0      1
L-SC       OX40N       88618.0      1
H-LC       X200        79134.0      1
X-SC       X40         76942.0      1
H-SC       MH          72687.0      1
H-MC       OX40N       71313.0      1
L-MC       XR          59088.0      1
L-LC       XY25        41982.0      1
M-LC       XY24        26983.0      1
H-MC       MH          10735.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
